# Neural Style Transfer with PyTorch
This notebook implements Neural Style Transfer using PyTorch and a pre-trained VGG19 model. The code will apply the artistic style of one image to the content of another image.

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Set up Google Drive paths
base_path = '/content/drive/MyDrive/Neural_Style_Transfer'
content_image_dir = f'{base_path}/content_images'
style_image_dir = f'{base_path}/style_images'
output_dir = f'{base_path}/output'

# Create directories if they don't exist
import os
for dir_path in [content_image_dir, style_image_dir, output_dir]:
    os.makedirs(dir_path, exist_ok=True)

## Install Required Dependencies

In [ ]:
!pip install torch torchvision pillow imageio

## Import Libraries and Set Up GPU/CPU Device

In [ ]:
import torch
import torch.optim as optim
from torchvision import transforms, models
from PIL import Image
import numpy as np
import os
import imageio

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')

## Define Helper Functions

In [ ]:
def preprocess_image(image_path, transform):
    image = Image.open(image_path).convert('RGB')
    return transform(image).unsqueeze(0).to(device)

def postprocess(tensor):
    image = tensor.to('cpu').clone().squeeze(0)
    image = image.detach().numpy().transpose(1, 2, 0)
    image = image * np.array([0.229, 0.224, 0.225]) + np.array([0.485, 0.456, 0.406])
    image = np.clip(image, 0, 1)
    return (image * 255).astype(np.uint8)

def gram_matrix(input_tensor):
    batch_size, c, h, w = input_tensor.size()
    features = input_tensor.view(batch_size * c, h * w)
    G = torch.mm(features, features.t())
    return G.div(batch_size * c * h * w)

def get_features(image, model, layer_indices):
    features = []
    x = image
    for idx, layer in enumerate(model):
        x = layer(x)
        if idx in layer_indices:
            features.append(x)
    return features

## Set Up Style Transfer Configuration and Load Images
**Note:** Before running this cell, upload your content and style images to the respective folders in Google Drive!

In [ ]:
# Configuration
content_path = os.path.join(content_image_dir, 'content1.jpeg')
style_path = os.path.join(style_image_dir, 'style3.jpeg')
iterations = 1000
content_weight = 1e2
style_weight = 1e7
lr = 0.05

# Transform
transform = transforms.Compose([
    transforms.Resize(512),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load images
content = preprocess_image(content_path, transform)
style = preprocess_image(style_path, transform)

# Load VGG19
vgg = models.vgg19(weights='IMAGENET1K_V1').features.to(device).eval()
for param in vgg.parameters():
    param.requires_grad_(False)

content_layers = [22]
style_layers = [1, 6, 11, 20, 29] # Keep these for now, but consider experimenting

content_features = get_features(content, vgg, content_layers)
style_features = get_features(style, vgg, style_layers)
style_grams = [gram_matrix(f) for f in style_features]

# Initial image
target = content.clone().requires_grad_(True)
optimizer = optim.LBFGS([target], lr=lr)


## Run Style Transfer Optimization

In [ ]:
intermediate_images = []
iteration = [0]

print('Starting optimization...')

while iteration[0] <= iterations:
    def closure():
        optimizer.zero_grad()
        target_content = get_features(target, vgg, content_layers)
        target_style = get_features(target, vgg, style_layers)
        target_grams = [gram_matrix(layer) for layer in target_style]

        content_loss = content_weight * torch.mean((target_content[0] - content_features[0])**2)
        style_loss = 0
        for tg, sg in zip(target_grams, style_grams):
            style_loss += torch.mean((tg - sg)**2)
        style_loss *= style_weight
        total_loss = content_loss + style_loss
        total_loss.backward()

        if iteration[0] % 100 == 0:
            print(f"Iteration {iteration[0]} | Content Loss: {content_loss.item():.2f}, Style Loss: {style_loss.item():.2f}")
            with torch.no_grad():
                img = postprocess(target)
                intermediate_images.append(img)
                Image.fromarray(img).save(os.path.join(output_dir, f'iter_{iteration[0]}.png'))
        iteration[0] += 1
        return total_loss
    optimizer.step(closure)

print('Optimization complete!')

## Save and Display Results

In [ ]:
# Save final image
final_image = postprocess(target)
Image.fromarray(final_image).save(os.path.join(output_dir, 'final.jpg'))

# Save progress GIF
if intermediate_images:
    imageio.mimsave(os.path.join(output_dir, 'progress.gif'),
                    [Image.fromarray(img) for img in intermediate_images], duration=500, loop=0)
print("Saved final outputs to:", output_dir)

In [ ]:
# Display the final generated image and progress
from IPython.display import Image as IPythonImage

# Display the final generated image
final_image_path = os.path.join(output_dir, 'final.jpg')
display(IPythonImage(filename=final_image_path))

# Display the progress GIF
if intermediate_images:
    progress_gif_path = os.path.join(output_dir, 'progress.gif')
    display(IPythonImage(filename=progress_gif_path))